In [1]:
import pandas as pd
import networkx as nx

In [2]:
mainData = pd.read_csv('../data/mainData.csv')
mainData.head()

,title,year,episode_name,season,episode_num,country,date,rating,votes,rank,name,type
0,$#*! My Dad Says,2010,Code Ed,1,4,USA,2010-10-14,6.8,72,6.899713,Alison (V) Brown,writer
1,$#*! My Dad Says,2010,Code Ed,1,4,USA,2010-10-14,6.8,72,6.899713,Justin (II) Halpern,creator
2,$#*! My Dad Says,2010,Code Ed,1,4,USA,2010-10-14,6.8,72,6.899713,David A. Katz,writer
3,$#*! My Dad Says,2010,Code Ed,1,4,USA,2010-10-14,6.8,72,6.899713,David Kohan,creator
4,$#*! My Dad Says,2010,Code Ed,1,4,USA,2010-10-14,6.8,72,6.899713,Michael Lisbe,written by


In [3]:
writers = mainData[mainData['type'].str.contains('writ').fillna(False)]
writers.head()

,title,year,episode_name,season,episode_num,country,date,rating,votes,rank,name,type
0,$#*! My Dad Says,2010,Code Ed,1,4,USA,2010-10-14,6.8,72,6.899713,Alison (V) Brown,writer
2,$#*! My Dad Says,2010,Code Ed,1,4,USA,2010-10-14,6.8,72,6.899713,David A. Katz,writer
4,$#*! My Dad Says,2010,Code Ed,1,4,USA,2010-10-14,6.8,72,6.899713,Michael Lisbe,written by
6,$#*! My Dad Says,2010,Code Ed,1,4,USA,2010-10-14,6.8,72,6.899713,Jesse Porter,writer
7,$#*! My Dad Says,2010,Code Ed,1,4,USA,2010-10-14,6.8,72,6.899713,Nate Reger,written by


In [4]:
len(writers.index)

142640

In [88]:
show_ratings = writers.groupby('title')['rank'].mean()
top_100 = list(show_ratings.sort_values(ascending=False).head(100).index)

In [89]:
grouped = writers[writers.title.isin(top_100)].groupby(['name'])

In [90]:
from itertools import combinations
from collections import Counter

output = []

for name, data in grouped:
    output.extend(set([x for x in combinations(data['title'],2) if x[0] != x[1]]))

output.sort()
c = Counter(output)

graph_builder = []
for x in c:
    graph_builder.append((x[0], x[1], {'weight': c[x]}))
graph_builder[0]

('Buffy the Vampire Slayer', 'The X Files', {'weight': 2})

In [91]:
G = nx.Graph()
G.add_edges_from(graph_builder)
for x in top_100:
    if x not in G.nodes():
        G.add_node(x)


In [92]:
import matplotlib.pyplot as plt
from math import exp

shows = writers[writers.title.isin(top_100)].groupby('title')

nx.draw_networkx(G, 
                 pos=nx.spring_layout(G), 
                 ax=None,
                 alpha=1,
                 nodelist = [show[0] for show in shows],
                 node_labels = list(degree.sort_values('degree', ascending=False).head(20).title),
                 node_size = [len(show[1]['name'].value_counts().index)*10 for show in shows],
                 node_color = [show[1]['rank'].mean() for show in shows],
                 vmin = min([show[1]['rank'].mean() for show in shows]),
                 vmax = max([show[1]['rank'].mean() for show in shows])
                )
plt.show()

In [140]:
import matplotlib.pyplot as plt
from math import exp

shows = writers[writers.title.isin(top_100)].groupby('title')
deg = nx.degree(G)

nx.draw_networkx(G, 
                 pos=nx.spring_layout(G), 
                 alpha=0.5,
                 nodelist = [show[0] for show in shows],
                 node_size = [exp(show[1]['rank'].mean())/10 for show in shows],
                 node_color = [G.degree(show[0]) for show in shows],
                 labels = dict(node_list),
                 with_labels = False,
                )
plt.show()

In [94]:
degree = pd.DataFrame.from_dict(nx.degree(G), orient='index').reset_index()
degree.columns = ['title', 'degree']
degree.sort_values('degree', ascending=False).head(20)


,title,degree
46,Angel,23
38,Buffy the Vampire Slayer,22
5,Lost,21
95,Fringe,19
27,The X Files,19
62,Firefly,16
68,Spartacus: Blood and Sand,16
82,Once Upon a Time,15
86,The Vampire Diaries,15
28,Supernatural,15


In [142]:
from collections import defaultdict
node_list = defaultdict(str)
high_degree = list(degree.sort_values('degree', ascending=False).head(20).title)
for x in [show[0] for show in shows]:
    if x in high_degree:
        node_list[x] = x
    else:
        node_list[x] = ''
    
pos=nx.spring_layout(G)

nx.draw_networkx(G, pos, alpha=0.5,
                 node_size = [exp(show[1]['rank'].mean())/10 for show in shows],
                 node_color = [G.degree(show[0]) for show in shows], nodelist=node_list.keys(), with_labels=False)
nx.draw_networkx_labels(G, pos, labels=dict(node_list))
plt.show()


In [103]:
nx.neighbors(G, '"Band of Brothers"')

['"The Pacific"', '"Justified"']

In [ ]:
degrees = pd.DataFrame.from_dict(G.degree(), orient='index').reset_index()
degrees.columns = ['Writer', 'degree']

In [ ]:
dd = defaultdict(lambda : defaultdict(int))
dd[show_1][show_2] += 1

In [ ]:
from math import exp
colors = writers.groupby('title')['rank'].mean().apply(lambda x: exp(x*10)).to_dict()
num_writers = 

In [ ]:
colors

In [151]:
shows = writers[writers.title.isin(top_100)].groupby('title')
[show[1]['rank'].mean() for show in shows]

[7.003883114854477,
 7.003182741169116,
 7.072844605296507,
 7.070614825655582,
 7.2769241180921425,
 7.470655801956319,
 7.270021627017134,
 7.034385122924979,
 7.043849643612731,
 7.153097593601197,
 7.015637723829485,
 7.076563761612914,
 7.57895523906097,
 7.014442192814791,
 7.131740275663853,
 7.005534019261218,
 7.005268123477754,
 7.060052278326723,
 7.183050450037953,
 7.036051654544879,
 7.037960248002147,
 7.003349876495887,
 7.036274674755052,
 7.633283659500646,
 7.020144475848395,
 7.02040883364909,
 7.018006656566343,
 7.13311352270747,
 7.233730567064744,
 7.344356264880803]